In [2]:
import os
import pyopencl as pcl
import numpy as np

# let's try to find available devices
platforms = pcl.get_platforms()
for p in platforms:
    devs = p.get_devices()
    for d in devs:
        print(d.name,d.type, pcl.device_type.to_string(d.type), d.global_mem_size / 10**9)

# let's select the AMD radeon card in this case
dev=None
for p in pcl.get_platforms():
    devs = p.get_devices()
    for d in devs:
        if pcl.device_type.to_string(d.type) == 'GPU' and (d.global_mem_size / 10**9) > 2.0:
            dev = d
            
# make the opencl context
# cntx = pcl.create_some_context()
cntx = pcl.Context( [dev])
queue = pcl.CommandQueue(cntx)

Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz 2 CPU 17.179869184
Iris Pro 4 GPU 1.610612736
AMD Radeon R9 M370X Compute Engine 4 GPU 2.147483648


In [3]:
ktest_cl_file = os.path.join('..', 'src', 'cl', 'kernel_tests.cl')
os.path.isfile(ktest_cl_file)

True

In [76]:
# build the kernel
with open(ktest_cl_file, 'r') as f:
    programs = pcl.Program(cntx, f.read()).build()
    f.seek(0)
    print(f.read())

__kernel void addem(__global float * a, __global float * b, __global float * c)
{

  int i = get_global_id(0);
  c[i] = a[i] + b[i];

}


__kernel void multiplyem(__global float * a, __global float * b, __global float * c)
{
  int i = get_global_id(0);
  c[i] = a[i] * b[i];
}

__kernel void testdot(__global float * a, __global float * b, __global float * c){
  int gid = get_global_id(0);
  c[gid] = dot(a[gid], b[gid]);
}

__kernel void test_rowaverage(__global float * in, __global float * out, const int nrows, const int ncols)
{
  float nrowsf = (float) nrows;
  for(int i = 0; i < nrows; i++){
    for (int j = 0; j < ncols; j++){
      out[j] += in[i * ncols + j];
      out[j] /= nrowsf;
    }
  }

}

__kernel void array_copy(__global float * in, __global float * out, const int nrows, const int ncols)
{
  int gid = get_global_id(0);
  for(int j = 0; j < ncols-1; j++){
    out[gid*ncols + j] = (float) (gid*ncols + j);
  }
}


__kernel void two_stage_reduce(__global float * in, __local f

/Users/antalek/anaconda/envs/ENVPy3/lib/python3.5/site-packages/pyopencl/cffi_cl.py:1476: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [77]:
test_arr = np.random.normal(size=(4,3)).astype(np.float32)
out_arr = np.zeros(test_arr.shape, dtype=np.float32)

In [78]:
test_arr

array([[-0.67914981,  1.23438013,  0.80597502],
       [-0.0764631 ,  0.35418963,  0.82736981],
       [-2.03588915,  1.11234605, -0.47378132],
       [-0.70324415, -1.47821152,  0.76152283]], dtype=float32)

In [79]:
test_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=test_arr)
out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=out_arr.nbytes)
copy_event = programs.array_copy(queue,
                                 test_arr.shape,
                                 None,
                                 test_buf,
                                 out_buf,
                                 np.int32(test_arr.shape[0]),
                                 np.int32(test_arr.shape[1]))

In [80]:
copy_event.wait()
pcl.enqueue_copy(queue, out_arr, out_buf).wait()

In [82]:
out_arr

array([[  0.00000000e+00,   1.00000000e+00,  -3.17465100e+11],
       [  3.00000000e+00,   4.00000000e+00,  -3.17465100e+11],
       [  6.00000000e+00,   7.00000000e+00,  -3.17465100e+11],
       [  9.00000000e+00,   1.00000000e+01,  -3.17465100e+11]], dtype=float32)

In [81]:
out_arr == test_arr

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False]], dtype=bool)

In [29]:
test_arr = np.ones(shape=(4,), dtype=np.float32)
test_arr2 = np.ones(shape=(4,), dtype=np.float32) * 2
out = np.zeros(shape=(1,), dtype=np.float32)

test_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=test_arr)
test_buf2 = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=test_arr2)
out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=out_arr.nbytes)

In [31]:
mydot_event = programs.test_dot2(queue,
                                 test_arr.shape,
                                 None,
                                 test_buf,
                                 test_buf2,
                                 out_buf,
                                 np.int32(test_arr.size))

In [32]:
mydot_event.wait()
pcl.enqueue_copy(queue, out, out_buf).wait()

In [33]:
out

array([ 8.], dtype=float32)

In [40]:
test_arr = np.ones(shape=(4,), dtype=np.float32) * 2
test_arr2 = np.ones(shape=(4,), dtype=np.float32) * 3
out = np.zeros(shape=(4,), dtype=np.float32)

test_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=test_arr)
test_buf2 = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=test_arr2)
out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=out_arr.nbytes)

In [41]:
pow_event = programs.powtest(queue,
                             test_arr.shape,
                             None,
                             test_buf,
                             test_buf2,
                             out_buf)

In [42]:
pow_event.wait()
pcl.enqueue_copy(queue, out, out_buf).wait()

In [43]:
out

array([ 8.,  8.,  8.,  8.], dtype=float32)

In [60]:
sig_x = np.array([[2,2,2],
                  [-2,-2,-2]], dtype=np.float32)
theta = np.array([0.5,0.5, 0.5], dtype=np.float32)
out = np.zeros(shape=(sig_x.shape[0],), dtype=np.float32)

In [61]:
out, theta

(array([ 0.,  0.], dtype=float32), array([ 0.5,  0.5,  0.5], dtype=float32))

In [62]:
sig_x_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=sig_x)
theta_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=theta)
out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=out.nbytes)

In [63]:
sig_event = programs.sig(queue,
                         sig_x.shape,
                         None,
                         sig_x_buf,
                         theta_buf,
                         out_buf,
                         np.int32(sig_x.shape[0]),
                         np.int32(sig_x.shape[1]))

In [64]:
sig_event.wait()
pcl.enqueue_copy(queue, out, out_buf).wait()

In [65]:
out

array([ 0.95257413,  0.04742588], dtype=float32)

In [66]:
def sig(X, theta):
    lin = X.dot(theta)
    sig = 1.0 / (1.0 + np.exp(-lin))
    return sig

In [69]:
sig(sig_x, theta)

array([ 0.95257413,  0.04742587], dtype=float32)

In [ ]:
all_k = programs.all_kernels()

In [ ]:
fk=all_k[0]

In [ ]:
fk.function_name

In [ ]:
# set up the buffers and arrays
a = np.ones(shape=(10, ), dtype=np.float32) * 3
# b = np.ones(shape=(10, ), dtype=np.float32) * 5
b = np.arange(0,10,1, dtype=np.float32)
c = np.zeros(shape=(10, ), dtype=np.float32)

In [ ]:
a, b, c

In [ ]:
a_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=a)
b_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=b)
c_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=c)

In [ ]:
# queue up the two kernels 
# add_event = programs.addem(queue,
#                            a.shape,
#                            None,
#                            a_buf, 
#                            b_buf,
#                            c_buf)

# multiply_event = programs.multiplyem(queue,
#                                      a.shape,
#                                      None,
#                                      c_buf,
#                                      a_buf,
#                                      b_buf)

dot_event = programs.testdot(queue,
                             a.shape,
                             None,
                             a_buf,
                             b_buf,
                             c_buf)

In [ ]:
# queue.finish()

In [ ]:
# add_event.wait()
# multiply_event.wait()

In [ ]:
pcl.enqueue_copy(queue, b, b_buf)
pcl.enqueue_copy(queue, c, c_buf)

In [ ]:
b

In [ ]:
c

In [ ]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]], dtype=np.float32)
x_avg = np.zeros(shape=(X.shape[1],), dtype=np.float32)

In [ ]:
c

In [ ]:
X_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=X)
x_avg_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=x_avg)

In [ ]:
x_avg

In [ ]:
row_avg_event = programs.test_rowaverage(queue,
                                         X.shape,
                                         None,
                                         X_buf, 
                                         x_avg_buf,
                                         np.int32(X.shape[0]),
                                         np.int32(X.shape[1]))

In [ ]:
pcl.enqueue_copy(queue, x_avg, x_avg_buf)

In [ ]:
x_avg

In [ ]:
X.mean(axis=0)

In [ ]:
X

In [ ]:
dev

In [ ]:
os.environ['PYOPENCL_COMPILER_OUTPUT'] = "1"
cntx = pcl.Context( [dev])
queue = pcl.CommandQueue(cntx, properties=pcl.command_queue_properties.PROFILING_ENABLE)
with open(ktest_cl_file, 'r') as f:
    programs = pcl.Program(cntx, f.read()).build()

In [ ]:
# z = np.array([1,2,3,4,5,6,7,8,9,10], dtype=np.float32)
z = np.ones(shape=(2**27,)).astype(np.float32)
z_out = np.zeros(shape=(1,), dtype=np.float32)
scratch = np.zeros(shape=z.shape, dtype=np.float32)

In [ ]:
(1024 * 1024 * 1024 )/10**9

In [ ]:
z.nbytes/1024/1024/1024

In [ ]:
z_out

In [ ]:
z_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=z)
z_out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=z_out.nbytes)
partial_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=scratch)
# partial_buf = pcl.LocalMemory(size=256*4)

In [ ]:
z.shape, z.size, z.nbytes/256, z_out.nbytes

In [ ]:
tsr = programs.all_kernels()[-4]

In [ ]:
tsr.function_name

In [ ]:
# reduction_event = programs.test_reduction_avg_global(queue,
#                                                       z.shape,
#                                                       (32,),
#                                                       z_buf,
#                                                       z_out_buf,
#                                                       partial_buf,
#                                                       np.int32(z.shape[0]))

# partial_buf = pcl.LocalMemory(size=256*4)
# reduction_event = programs.test_reduction_avg(queue,
#                                               z.shape,
#                                               None,
#                                               z_buf,
#                                               z_out_buf,
#                                               partial_buf,
#                                               np.int32(z.shape[0]))

partial_buf = pcl.LocalMemory(256*4)
i = 64
z_out = np.zeros(shape=(i,), dtype=np.float32)
z_out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=z_out.nbytes)

reduction_event = tsr(queue,
                      (256*i,),
                      (256,),
                      z_buf,
                      partial_buf,
                      z_out_buf,
                      np.int32(z.shape[0]))

# reduction_event = programs.two_stage_reduce(queue,
#                                             (256,),
#                                             (256,),
#                                             z_buf,
#                                             partial_buf,
#                                             z_out_buf,
#                                             np.int32(z.shape[0]))

In [ ]:
tsr.get_work_group_info(pcl.kernel_work_group_info.PREFERRED_WORK_GROUP_SIZE_MULTIPLE, dev)

In [ ]:
dev.global_mem_size

In [ ]:
dev.max_work_group_size

In [ ]:
%%time
reduction_event.wait()

In [ ]:
%%time
pcl.enqueue_copy(queue, z_out, z_out_buf).wait()

In [ ]:
%%time
z_out.sum()

In [ ]:
%%time
z.sum()

In [ ]:
np.float32((z.sum())) == z_out[0]

In [ ]:
z.shape[0]/4096

In [ ]:
z_out

In [ ]:
z.sum()/256

In [ ]:
z_out.shape[0] / 256

In [ ]:
(z_out == 256.0).sum()

In [ ]:
z.sum()/z_out[0]

In [ ]:
nrows = int(2**9)
y = np.random.normal(size = (nrows, 6), loc=10.0).astype(np.float32)
y_row_avg = np.zeros(shape=(6,)).astype(np.float32)

In [ ]:
y.nbytes

In [ ]:
y_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=y)
y_out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=y_row_avg.nbytes)
partial_sum_buf = pcl.LocalMemory(size=y.nbytes)

In [ ]:
row_reduction_event = programs.test_reduction_avg_matrix(queue,
                                                         y.shape,
                                                         None,
                                                         y_buf,
                                                         y_out_buf,
                                                         partial_sum_buf,
                                                         np.int32(y.shape[0]),
                                                         np.int32(y.shape[1]))

In [ ]:
row_reduction_event.wait()

In [ ]:
pcl.enqueue_copy(queue, y_row_avg, y_out_buf, wait_for=[row_reduction_event])

In [ ]:
y_row_avg

In [ ]:
y.mean(axis=0), y_row_avg

In [ ]:
y.nbytes * 2

In [ ]:
y.mean(axis=0)

In [ ]:
# from: https://github.com/pyopencl/pyopencl/blob/master/examples/benchmark.py

In [ ]:
from __future__ import print_function
from __future__ import absolute_import
import pyopencl as cl
import numpy
import numpy.linalg as la
import datetime
from time import time

data_points = 2**24 # ~8 million data points, ~32 MB data
workers = 2**6 # 256 workers, play with this to see performance differences
               # eg: 2**0 => 1 worker will be non-parallel execution on gpu
               # data points must be a multiple of workers

a = numpy.random.rand(data_points).astype(numpy.float32)
b = numpy.random.rand(data_points).astype(numpy.float32)
c_result = numpy.empty_like(a)

# Speed in normal CPU usage
time1 = time()
c_temp = (a+b) # adds each element in a to its corresponding element in b
c_temp = c_temp * c_temp # element-wise multiplication
c_result = c_temp * (a/2.0) # element-wise half a and multiply
time2 = time()

print("Execution time of test without OpenCL: ", time2 - time1, "s")


for platform in cl.get_platforms():
    for device in platform.get_devices():
        print("===============================================================")
        print("Platform name:", platform.name)
        print("Platform profile:", platform.profile)
        print("Platform vendor:", platform.vendor)
        print("Platform version:", platform.version)
        print("---------------------------------------------------------------")
        print("Device name:", device.name)
        print("Device type:", cl.device_type.to_string(device.type))
        print("Device memory: ", device.global_mem_size//1024//1024//1024, 'GB')
        print("Device max clock speed:", device.max_clock_frequency, 'MHz')
        print("Device compute units:", device.max_compute_units)
        print("Device max work group size:", device.max_work_group_size)
        print("Device max work item sizes:", device.max_work_item_sizes)

        # Simnple speed test
        ctx = cl.Context([device])
        queue = cl.CommandQueue(ctx, 
                properties=cl.command_queue_properties.PROFILING_ENABLE)

        mf = cl.mem_flags
        a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
        b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
        dest_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)

        prg = cl.Program(ctx, """
            __kernel void sum(__global float *a,
            __global float *b, __global float *c)
            {
                        int gid = get_global_id(0);
                        float a_temp;
                        float b_temp;
                        float c_temp;
                        a_temp = a[gid]; // my a element (by global ref)
                        b_temp = b[gid]; // my b element (by global ref)
                        
                        c_temp = a_temp+b_temp; // sum of my elements
                        c_temp = c_temp * c_temp; // product of sums
                        c_temp = c_temp * (a_temp/2.0f); // times 1/2 my a
                        c[gid] = c_temp; // store result in global memory
                }
                """).build()

        global_size=(data_points,1)
        local_size=(workers,)
        preferred_multiple = cl.Kernel(prg, 'sum').get_work_group_info( \
            cl.kernel_work_group_info.PREFERRED_WORK_GROUP_SIZE_MULTIPLE, \
            device)

        print("Data points:", data_points)
        print("Workers:", workers)
        print("Preferred work group size multiple:", preferred_multiple)

        if (workers % preferred_multiple):
            print("Number of workers not a preferred multiple (%d*N)." \
                    % (preferred_multiple))
            print("Performance may be reduced.")

        exec_evt = prg.sum(queue, global_size, None, a_buf, b_buf, dest_buf)
        exec_evt.wait()
        elapsed = 1e-9*(exec_evt.profile.end - exec_evt.profile.start)

        print("Execution time of test: %g s" % elapsed)

        c = numpy.empty_like(a)
        cl.enqueue_copy(queue, c, dest_buf).wait()
        equal = numpy.all( c == c_result)

        if not equal:
                print("Results doesn't match!!")
        else:
                print("Results OK")